# 음주 여부에 따라 건강검진 수치 차이가 있을까?

# 신장과 허리둘레의 크기는 체중과 상관관계가 있을까?

# 라이브러리 로드

In [2]:
# 분석에 사용할 pandas, 수치계산에 사용할 numpy, 시각화에 사용할 seaborn을 불러 옵니다
# 구 버전의 주피터 노트북에서는 %matplotlib inline 설정이 되어야 노트북 안에서 그래프를 시각화 합니다. 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 한글 폰트 설정

In [3]:
# Google Colab  사용시 아래 주석을 풀고 폰트 설정을 합니다.
# 로컬 아나콘다 사용시에는 그대로 주석처리 해놓으시면 됩니다. 
# 나눔고딕 설치
# 이 코드를 사용시 아래에 있는 폰트를 로드할 경우 Colab에서는 오류가 발생하니 아래에 있는 폰트 설정을 꼭 주석처리 할것
# !apt -qq -y install fonts-nanum > /dev/null

# import matplotlib.font.manager as fm

# fontpath = '/user/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# font = fm.FontProperties(fname=fontpath, size=9)
# fm._rebuild()

# colab의 한글 폰트 설정
# plt.rc('font', family='NanumGothic')
# 마이너스 폰트 깨지는 문제에 대한 대처
# plt.rc("axes", unicode_minus=False)

In [4]:
# 한글폰트를 설정해 주지 않으면 그래프 상에서 한글이 깨져 보입니다
# 한글이 출력될수 있도록 폰트 설정을 해줍니다
# 운영체제별 설명을 위해 로드 합니다

import os
if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")
    
# 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc("axes", unicode_minus=False)

In [5]:
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다

%config InlineBackend.figure_format = 'retina'

# 데이터 불러오기

In [1]:
# 다운로드 받은 파일을 판다스의 read_csv를 통해 읽어 옵니다
# 파일을 읽어 온 후 shape로 행과 열의 수를 출력합니다.

df = pd.read_csv("건강검진정보.csv")

NameError: name 'pd' is not defined